## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-08-18-54-28 +0000,nypost,"Mom, 6-year-old twins shot dead in mansion ami...",https://nypost.com/2025/12/08/us-news/arkansas...
1,2025-12-08-18-50-58 +0000,nyt,FBI Agents Sue Patel After Being Fired Over Kn...,https://www.nytimes.com/2025/12/08/us/politics...
2,2025-12-08-18-50-15 +0000,nyt,Jared Kushner Is Part of Paramount’s Bid for W...,https://www.nytimes.com/2025/12/08/business/me...
3,2025-12-08-18-47-11 +0000,nypost,Danika Troy’s killers lured daughter into wood...,https://nypost.com/2025/12/08/us-news/danika-t...
4,2025-12-08-18-47-07 +0000,bbc,Sweaty Betty in new dispute over ad slogans,https://www.bbc.com/news/articles/cj9yz0xmvk2o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2332/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
50,trump,45
27,warner,18
174,deal,16
46,new,16
53,netflix,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
107,2025-12-08-15-28-04 +0000,nypost,Paramount Skydance launches hostile bid for Wa...,https://nypost.com/2025/12/08/business/paramou...,146
181,2025-12-08-10-44-44 +0000,bbc,Trump says $72bn Netflix-Warner Bros deal 'cou...,https://www.bbc.com/news/articles/cn815egjqjpo...,113
139,2025-12-08-14-06-59 +0000,nyt,Trump Looms Over Netflix’s Deal for Warner Bro...,https://www.nytimes.com/2025/12/08/business/de...,113
235,2025-12-08-02-40-46 +0000,wapo,Trump says he will be ‘involved’ in vetting Ne...,https://www.washingtonpost.com/business/2025/1...,111
248,2025-12-08-00-23-00 +0000,wsj,Trump Says Netflix-Warner Deal ‘Could Be a Pro...,https://www.wsj.com/business/media/trump-raise...,98


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
107,146,2025-12-08-15-28-04 +0000,nypost,Paramount Skydance launches hostile bid for Wa...,https://nypost.com/2025/12/08/business/paramou...
6,52,2025-12-08-18-44-00 +0000,wsj,Supreme Court Appears Poised to Give President...,https://www.wsj.com/politics/policy/supreme-co...
151,47,2025-12-08-13-20-35 +0000,nypost,‘Disappointed’ Trump claims Zelensky ‘hasn’t y...,https://nypost.com/2025/12/08/world-news/disap...
130,42,2025-12-08-14-51-32 +0000,nypost,New winter storm stretching as far south as No...,https://nypost.com/2025/12/08/us-news/new-wint...
241,41,2025-12-08-01-54-25 +0000,nypost,JPMorgan Chase’s Jamie Dimon claims AI will no...,https://nypost.com/2025/12/07/business/jamie-d...
225,38,2025-12-08-04-15-18 +0000,nyt,"Despite Trump’s Tariffs, China’s Trade Surplus...",https://www.nytimes.com/2025/12/07/business/ch...
201,37,2025-12-08-09-28-00 +0000,wsj,Japan Is Out Spending. Bond Markets Seem Nervo...,https://www.wsj.com/economy/japan-is-out-spend...
282,34,2025-12-07-20-18-00 +0000,nypost,SL Green chief expects thriving NYC office ma...,https://nypost.com/2025/12/07/business/sl-gree...
28,31,2025-12-08-18-00-42 +0000,nypost,Students form human swastika on Calif. high sc...,https://nypost.com/2025/12/08/us-news/human-sw...
79,30,2025-12-08-16-28-03 +0000,wapo,European leaders meet Zelensky amid Trump pres...,https://www.washingtonpost.com/world/2025/12/0...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
